<p style="font-family: Comic San ML"><font size=3>Topic modeling is an unsupervised technique that intends to analyze large volumes of text data by clustering the documents into groups. In the case of topic modeling, the text data do not have any labels attached to it. Rather, topic modeling tries to group the documents into clusters based on similar characteristics.</font></p>

https://stackabuse.com/python-for-nlp-topic-modeling/

<h1 style="color:#4B0082;font-family:Comic Sans MS;">Latent Dirichlet Allocation (LDA)</h1>

1. Documents that have similar words usually have the same topic = Documents are probability distributions over latent topics
2. Documents that have groups of words frequently occuring together usually have the same topic = Topics are probability distributions over words

In [8]:
import pandas as pd
import numpy as np

dataset = pd.read_csv(r'C:/Users/gabid/meus_desastres/NLTK/sentimentos.csv', sep=";")
dataset.head()

,sentimento,text
0,neg,Retorno de alunos às aulas presenciais deverá ...
1,neg,O maior risco são os ônibus eo metro e desde ...
2,neg,"olá a todos, venho aqui encarecidamente pedir ..."
3,neg,Justiça nega pedido de retorno das aulas prese...
4,pos,n aguento mais o EAD


In [11]:
dataset['text'][10]

'A minha escola é só de ensino médio e tem muitas salas vazias, e a direção irá separar vários alunos por turma e por em salas separadas, as minhas aulas estão previstas pra começar dia 10/08'

In [22]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')
#create vocabulary
from sklearn.feature_extraction.text import CountVectorizer
#creating a document-term matrix
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words=stopwords)
doc_term_matrix = count_vect.fit_transform(dataset['text'].values.astype('U'))

In [25]:
doc_term_matrix
#matrix com numero_de_amostras x numero_de_palavras

<26x41 sparse matrix of type '<class 'numpy.int64'>'
	with 111 stored elements in Compressed Sparse Row format>

In [35]:
from sklearn.decomposition import LatentDirichletAllocation
#The parameter n_components specifies the number of categories, or topics, that we want our text to be divided into.
LDA = LatentDirichletAllocation(n_components=2, random_state=12)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=12, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [36]:
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

#10 palavras aleatorias

agora
aulas
ônibus
escolas
pode
adultos
volta
agora
faculdade
criança


In [37]:
#10 words with the highest probability for the first topic
first_topic = LDA.components_[0]
#Once sorted, the 10 words with the highest probabilities will now belong to the last 10 indexes of the array.
top_topic_words = first_topic.argsort()[-10:]

In [38]:
top_topic_words

array([12, 16, 39, 13, 34, 22,  8, 30,  2,  4], dtype=int64)

In [39]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

escola
ficar
volta
escolas
retorno
lotados
dia
presenciais
alunos
aulas


In [40]:
#Let's print the 10 words with highest probabilities for all topics:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['escola', 'ficar', 'volta', 'escolas', 'retorno', 'lotados', 'dia', 'presenciais', 'alunos', 'aulas']


Top 10 words for topic #1:
['pode', 'esposa', 'filho', 'papel', 'escolas', 'risco', 'gente', 'dificuldade', 'agora', 'ano']




In [41]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape
#(numero de documentos, colunas de cada documento)

(26, 2)

In [42]:
topic_values

array([[0.92004038, 0.07995962],
       [0.86081269, 0.13918731],
       [0.91452977, 0.08547023],
       [0.89766282, 0.10233718],
       [0.74295918, 0.25704082],
       [0.74295918, 0.25704082],
       [0.07968269, 0.92031731],
       [0.80306846, 0.19693154],
       [0.25429385, 0.74570615],
       [0.1092917 , 0.8907083 ],
       [0.92894398, 0.07105602],
       [0.08360144, 0.91639856],
       [0.89046753, 0.10953247],
       [0.8595828 , 0.1404172 ],
       [0.94209867, 0.05790133],
       [0.498334  , 0.501666  ],
       [0.38168368, 0.61831632],
       [0.25428564, 0.74571436],
       [0.07315335, 0.92684665],
       [0.8282473 , 0.1717527 ],
       [0.91469014, 0.08530986],
       [0.0736568 , 0.9263432 ],
       [0.10766324, 0.89233676],
       [0.25279615, 0.74720385],
       [0.17607882, 0.82392118],
       [0.90790171, 0.09209829]])

In [43]:
dataset['text'] = topic_values.argmax(axis=1)

In [45]:
dataset.head(10)

,sentimento,text,Topic
0,neg,Retorno de alunos às aulas presenciais deverá ...,0
1,neg,O maior risco são os ônibus eo metro e desde ...,0
2,neg,"olá a todos, venho aqui encarecidamente pedir ...",0
3,neg,Justiça nega pedido de retorno das aulas prese...,0
4,pos,n aguento mais o EAD,0
5,neg,É o certo né. EaD é um saco mas é mais seguro,0
6,neg,Impossivel eu mandar meu filho de 4 anos para ...,1
7,pos,"Absurdo , as escolas tem que se preparar para ...",0
8,neg,"Eu por exemplo, tenho professores que já tem i...",1
9,neg,"Já estamos quase em Agôsto, o ano já está term...",1


<h1 style="color:#4B0082;font-family:Comic Sans MS;">Non-Negative Matrix Factorization (NMF)</h1>

In [46]:
import pandas as pd
import numpy as np

data_set = pd.read_csv(r'C:/Users/gabid/meus_desastres/NLTK/sentimentos.csv', sep=";")
data_set.head()

,sentimento,text
0,neg,Retorno de alunos às aulas presenciais deverá ...
1,neg,O maior risco são os ônibus eo metro e desde ...
2,neg,"olá a todos, venho aqui encarecidamente pedir ..."
3,neg,Justiça nega pedido de retorno das aulas prese...
4,pos,n aguento mais o EAD


In [48]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=stopwords)
doc_term_matrix = tfidf_vect.fit_transform(data_set['text'].values.astype('U'))

In [49]:
#creating uma matrix de probabilidade de todas as palavras do documento
from sklearn.decomposition import NMF

nmf = NMF(n_components=2, random_state=12)
nmf.fit(doc_term_matrix )

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=2, random_state=12, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [50]:
import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

alunos
alunos
pandemia
papel
filho
ead
pra
filhos
risco
pra


In [51]:
first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]

In [52]:
top_topic_words

array([ 5,  0, 35, 36, 18,  3, 34,  4, 13, 30], dtype=int64)

In [53]:
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

colocar
adultos
risco
setembro
filhos
ano
retorno
aulas
escolas
presenciais


In [54]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['colocar', 'adultos', 'risco', 'setembro', 'filhos', 'ano', 'retorno', 'aulas', 'escolas', 'presenciais']


Top 10 words for topic #1:
['ficar', 'escola', 'muitas', 'dia', 'faculdade', 'volta', 'alunos', 'gente', 'aulas', 'dificuldade']




In [57]:
topic_values = nmf.transform(doc_term_matrix)
data_set['Topic'] = topic_values.argmax(axis=1)
data_set.head(10)

,sentimento,text,Topic
0,neg,Retorno de alunos às aulas presenciais deverá ...,0
1,neg,O maior risco são os ônibus eo metro e desde ...,1
2,neg,"olá a todos, venho aqui encarecidamente pedir ...",1
3,neg,Justiça nega pedido de retorno das aulas prese...,0
4,pos,n aguento mais o EAD,0
5,neg,É o certo né. EaD é um saco mas é mais seguro,0
6,neg,Impossivel eu mandar meu filho de 4 anos para ...,1
7,pos,"Absurdo , as escolas tem que se preparar para ...",0
8,neg,"Eu por exemplo, tenho professores que já tem i...",1
9,neg,"Já estamos quase em Agôsto, o ano já está term...",0
